<a href="https://colab.research.google.com/github/RudyMartin/dsai-2024/blob/main/MVPS/Camp-Rock-Paper-Scissors/team_GG_AI/gestures_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygame

In [ ]:
import pygame
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from threading import Thread

# Initialize Pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Create the screen object
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Head Gesture Control Game')

# Define the player object by extending pygame.sprite.Sprite
# The surface drawn on the screen is now an attribute of 'player'
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.Surface((50, 50))
        self.surf.fill(WHITE)
        self.rect = self.surf.get_rect()

    def update(self, gesture):
        if gesture == 0:  # Up
            self.rect.move_ip(0, -5)
        elif gesture == 1:  # Down
            self.rect.move_ip(0, 5)
        elif gesture == 2:  # Right
            self.rect.move_ip(5, 0)
        elif gesture == 3:  # Left
            self.rect.move_ip(-5, 0)

        # Keep player on the screen
        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

# Initialize player
player = Player()

# Create groups to hold sprite objects
all_sprites = pygame.sprite.Group()
all_sprites.add(player)

# Load the trained model
model = load_model('head_gesture_model.keras')

# Function to process webcam input and update gesture
def gesture_recognition():
    global gesture
    cap = cv2.VideoCapture(0)
    while running:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (64, 64))
        input_frame = np.expand_dims(resized, axis=0)
        input_frame = np.expand_dims(input_frame, axis=-1)
        prediction = model.predict(input_frame)
        gesture = np.argmax(prediction)
        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Global variable to store the gesture
gesture = -1
running = True

# Start the gesture recognition thread
gesture_thread = Thread(target=gesture_recognition)
gesture_thread.start()

# Game loop
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update the player sprite based on the gesture
    player.update(gesture)

    # Fill the screen with black
    screen.fill(BLACK)

    # Draw all sprites
    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)

    # Update the display
    pygame.display.flip()

    # Ensure the program maintains a rate of 30 frames per second
    clock.tick(30)

# Clean up
running = False
gesture_thread.join()
pygame.quit()


pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


ValueError: File not found: filepath=head_gesture_model.keras. Please ensure the file is an accessible `.keras` zip file.